#### 아래 내용은 Intel® AI for Youth 프로그램을 참고하여
BrainAI와 BrainAI Coach Network에서 개발한 내용입니다.<br> 
상업적 사용은 불가하며 교육기관 및 학교에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: 다중자동분류 테스트
구글 Teachable Machine에서 개발한 AI 모델을 활용하여 M&M 초콜릿을 6가지 Color로 자동 분류하는 파이썬 프로그램 작성하기

### Task 1. 필요 라이브러리 불러오기

In [1]:
import cv2
import numpy as np
import time
import tensorflow.keras
from tensorflow.keras.models import model_from_json

import serial
import serial.tools.list_ports
print("프로젝트 수행에 필요한 라이브러리를 모두 불러왔습니다.")

프로젝트 수행에 필요한 라이브러리를 모두 불러왔습니다.


### Task 2. Teachable Machine에서 개발한 모델(model)과 클래스(label) 불러오기 함수 작성

In [2]:
def loadModel(): 
    model = tensorflow.keras.models.load_model('model/keras_model.h5', compile = False) 
    labels = []
    file = open("model/labels.txt", "r")
    for x in file:
        labels.append(x.rstrip('\n'))
    print('labels = ', labels)
    file.close()
    return model, labels

### Task 3. 새로운 이미지 데이터(M&M 초콜릿) Color 예측하기 함수 작성

In [3]:
def ReadPicture(cropped, model):   
    
    input_width = 224
    input_height = 224

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    normalized_image = (imgRGB.astype(np.float32) / 127.0) - 1
    batch = normalized_image.reshape(1, input_height, input_width, 3)

    prediction = model.predict(batch) 

    class_id = np.argmax(prediction)
    score = np.max(prediction) 
    
    return class_id, score

### Task 4. 마이크로비트 연결 포트 자동 찾기 함수 작성

In [4]:
def startSerial(serial_on):
    ser = None
    if serial_on == True:
        ports = serial.tools.list_ports.comports()
        com = ''
        print('\n -- Microbit --')
        for port, desc, hwid in sorted(ports):
            if 'USB' in desc:
                com = port # 포트 값을 가져와 com변수에 저장
        if com != '':
            print('Microbit USB detected: ', com)
            ser = serial.Serial(com, 115200 ,  # 마이크로비트 전송 속도와 일치
                                timeout=0, 
                                parity=serial.PARITY_NONE, 
                                rtscts=0) 
        else:
            print('No Microbit USB detected')
    return ser

### Task 5. 마이크로비트에 시리얼 통신으로 명령 보내기 함수 작성
마이크로비트 코드의 전송 속도(115200)를 확인한다.

In [5]:
def SerialSendCommand(cmd, ser, serial_on):
    if serial_on == True:  #serial 통신을 사용한다면
        cmd = str(cmd) 
        cmd  = cmd + '\n'
        cmd = str.encode(cmd) 
        ser.write(cmd)

### Task 6. Main 함수 작성

In [14]:
def Main():
    video_capture = cv2.VideoCapture(video)
    tic = time.time()
    
    # ---- 영상안에 p 눌러 시작하는 텍스트 출력 ---- #    
    color_p  = (0, 0, 255)
    text = "Press [p] to start"
    print(text)
    color_result = (255, 0, 0)
    result = "Wait... "
        
    # ---- 불러온 영상 크기 값 저장 ---- #    
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    
    # ---- 영상에서 매개 변수 top, bottom 값을 이용하여 관심 영역 설정 ---- #   
    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)
    
    # ---- MODE를 이용하여 실행, 멈춤을 컨트롤 ---- #
    MODE_PAUSE = -1 
    MODE_START = 0
    mode_status = MODE_PAUSE

    # ---- 시리얼 통신 값 저장 ----#
    ser = startSerial(serial_on)
    # ---- Teachable Machine에서 개발한 모델(model)과 클래스(label) 불러오기 실행 ----#
    model, labels = loadModel()
        
    while(True):
        grabbed, frame = video_capture.read()
        frame = cv2.flip(frame, 1)
        
        # ---- 관심 영역을 변수 cropped에 저장  ---- #
        cropped = frame[y1:y2, x1:x2]
        # ---- 영상에 관심 영역 박스 그리기  ---- #
        cv2.rectangle(frame, (x1,y1), (x2,y2), color_p, 2)
        # ---- 영상에 텍스트(text변수에 저장된 값) 출력 하기  ---- #
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, color_p, 1)
        # ---- 영상에 텍스트(result변수에 저장된 값) 출력 하기  ---- #
        cv2.putText(frame, result, (350,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color_result, 2)  

        # ---- 모델을 활용한 예측 실행  ---- #
        if mode_status == MODE_START and time.time() - tic > timeDelay: # 3초마다 예측 실행
            cv2.imshow("cropped", cropped)
            # 예측 실행하여 결과 저장하기
            class_id, score = ReadPicture(cropped, model)   
            print("Class:", class_id)
            print("Confidence:", score)

            # 결과 출력하기
            result = labels[class_id][2:] + ' ' + str(int(score*100)) + '%'
            result_text = 'AI 예측 결과는  ' + labels[class_id][2:] + ' ' + str(int(score*100)) + '%' + '입니다'
            print(result_text)
            tic = time.time()
            
            # ---- 60%이상 정확하면 시리얼 통신을 통해 다중자동분류기에 있는 마이크로비트에 명령 보내기 ---- #                
            if score >= .6:
                cmd = labels[class_id][0]
                SerialSendCommand(cmd, ser, serial_on)
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), color_p, 2)      
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, color_p, 2)  
        cv2.imshow("Frame", frame) 
        
        key = cv2.waitKey(1)
           
        if key == ord("q"):
            break
             
        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE       
                color = (0, 0, 255)
                text = "Press 'p' to start"

            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                color = (0, 255, 0)
                text = "Press 'p' to pause"

        elif key != -1:
            cmd = chr(key)
            text = cmd
            SerialSendCommand(cmd, ser, serial_on)

    # ---- 프로그램 종료시 마이크로비트 사용 포트를 닫는다. ---- #                
    if serial_on == True:
        ser.close()
        
    video_capture.release()
    cv2.destroyAllWindows()

### Task 7. 설정 값 변경 및 프로그램 실행

In [15]:
video = 0 #노트북 built in 웹캠, 외부 연결 웹캠인 경우 1로 수정
timeDelay = 3

#---- 관심 영역 설정 값 ----#
top = .65
bottom = .4

# 마이크로비트 없이 실행 가능: serial_on = False, 다중자동분류기 마이크로비트에 연결하여 작동시: serial_on = True
serial_on = True 

#---- 프로그램 실행 ----#
if __name__ == '__main__': 
    Main()

Press [p] to start

 -- Microbit --
Microbit USB detected:  COM3
labels =  ['0 None', '1 Red', '2 Orange', '3 Yellow', '4 Green', '5 Blue', '6 Brown']
Class: 0
Confidence: 0.85002166
AI 예측 결과는  None 85%입니다
Class: 0
Confidence: 0.6863565
AI 예측 결과는  None 68%입니다
